In [ ]:
!pip install facebook-sdk


In [ ]:
import facebook as fb
import os
import time

In [ ]:
access_token = "#"

In [ ]:

autofb = fb.GraphAPI(access_token)

In [ ]:


def select_image_and_caption(image_folder, caption_folder):
    images = [f for f in os.listdir(image_folder) if f.endswith(('png', 'jpg', 'jpeg'))]
    captions = [f for f in os.listdir(caption_folder) if f.endswith('.txt')]

    matched_files = []

    for image in images:
        image_name = os.path.splitext(image)[0]
        caption_file = f"{image_name}.txt"

        if caption_file in captions:
            with open(os.path.join(caption_folder, caption_file), 'r') as caption_f:
                caption = caption_f.read().strip()
            image_path = os.path.join(image_folder, image)
            matched_files.append((image_path, caption))

    return matched_files

image_folder = '/content/images'
caption_folder = '/content/captions'


matched_files = select_image_and_caption(image_folder, caption_folder)

for image_path, caption in matched_files:
    selected_image = image_path
    selected_caption = caption

    print(f"Posting Image: {selected_image}")
    print(f"Caption: {selected_caption}")

    autofb.put_photo(open(selected_image, "rb"), message=selected_caption)
    time.sleep(300)


integration


In [ ]:
import requests

# Instagram API credentials
instagram_access_token = "#"
instagram_account_id = "#"

# Facebook API credentials
facebook_access_token = "#"
facebook_page_id = "#"

# Define Instagram Graph API endpoints
instagram_media_upload_url = f"#"
instagram_media_publish_url = f"#"

# Define Facebook Graph API endpoint
facebook_post_url = f"#"


def post_to_instagram(image_url, caption):
    """Upload and publish a photo to Instagram."""
    media_upload_data = {
        "image_url": image_url,
        "caption": caption,
        "access_token": instagram_access_token
    }
    media_response = requests.post(instagram_media_upload_url, data=media_upload_data)

    if media_response.status_code != 200:
        print(f"Instagram media upload failed: {media_response.text}")
        return

    media_id = media_response.json()["id"]
    print(f"Instagram media uploaded successfully. Media ID: {media_id}")

    media_publish_data = {
        "creation_id": media_id,
        "access_token": instagram_access_token
    }
    publish_response = requests.post(instagram_media_publish_url, data=media_publish_data)

    if publish_response.status_code != 200:
        print(f"Instagram post publish failed: {publish_response.text}")
    else:
        print(f"Instagram post published successfully. Response: {publish_response.text}")


def post_to_facebook(image_url, caption):
    """Post a photo to Facebook page."""
    facebook_post_data = {
        "url": image_url,
        "caption": caption,
        "access_token": facebook_access_token
    }
    post_response = requests.post(facebook_post_url, data=facebook_post_data)

    if post_response.status_code != 200:
        print(f"Facebook post failed: {post_response.text}")
    else:
        print(f"Facebook post published successfully. Response: {post_response.text}")


def post_to_both(image_url, caption):
    """Simultaneously post to Instagram and Facebook."""
    print("Posting to Instagram...")
    post_to_instagram(image_url, caption)

    print("Posting to Facebook...")
    post_to_facebook(image_url, caption)


# Example usage
image_url = "https://photographylife.com/wp-content/uploads/2023/05/Nikon-Z8-Official-Samples-00002.jpg"
caption = "Sample caption for both Instagram and Facebook!"

post_to_both(image_url, caption)

Posting to Instagram...
Instagram media uploaded successfully. Media ID: 17852203812283344
Instagram post published successfully. Response: {"id":"17962493225801351"}
Posting to Facebook...
Facebook post published successfully. Response: {"id":"122107340060533711","post_id":"460354680485224_122107340138533711"}


final integration ig



In [ ]:
import os
import time
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

# Instagram and Facebook API credentials
instagram_access_token = "#"
instagram_account_id = "#"
facebook_access_token = "#"
facebook_page_id = "#"

# Imgur API credentials
imgur_client_id = "#"  # Replace with your Imgur client ID
imgur_upload_url = "#"

# Define Instagram and Facebook Graph API endpoints
instagram_media_upload_url = f"https://graph.facebook.com/v20.0/{instagram_account_id}/media"
instagram_media_publish_url = f"https://graph.facebook.com/v20.0/{instagram_account_id}/media_publish"
facebook_post_url = f"https://graph.facebook.com/v20.0/{facebook_page_id}/photos"

# Initialize BLIP model and processor for caption generation
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")


def generate_caption(image_path):
    """Generate a caption for the given image using the BLIP model."""
    image = Image.open(image_path)
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    outputs = model.generate(pixel_values)
    caption = processor.decode(outputs[0], skip_special_tokens=True)
    return caption


def upload_to_imgur(image_path):
    """Upload an image to Imgur and return the public URL."""
    headers = {
        "Authorization": f"Client-ID {imgur_client_id}"
    }

    with open(image_path, 'rb') as image_file:
        files = {'image': image_file}
        response = requests.post(imgur_upload_url, headers=headers, files=files)

    if response.status_code == 200:
        image_url = response.json()['data']['link']
        print(f"Image uploaded to Imgur successfully. URL: {image_url}")
        return image_url
    else:
        print(f"Failed to upload image to Imgur: {response.text}")
        return None


def post_to_instagram(image_url, caption):
    """Upload and publish a photo to Instagram."""
    media_upload_data = {
        "image_url": image_url,
        "caption": caption,
        "access_token": instagram_access_token
    }
    media_response = requests.post(instagram_media_upload_url, data=media_upload_data)

    if media_response.status_code != 200:
        print(f"Instagram media upload failed: {media_response.text}")
        return

    media_id = media_response.json()["id"]
    print(f"Instagram media uploaded successfully. Media ID: {media_id}")

    media_publish_data = {
        "creation_id": media_id,
        "access_token": instagram_access_token
    }
    publish_response = requests.post(instagram_media_publish_url, data=media_publish_data)

    if publish_response.status_code != 200:
        print(f"Instagram post publish failed: {publish_response.text}")
    else:
        print(f"Instagram post published successfully. Response: {publish_response.text}")


def post_to_facebook(image_url, caption):
    """Post a photo to Facebook page."""
    facebook_post_data = {
        "url": image_url,
        "caption": caption,
        "access_token": facebook_access_token
    }
    post_response = requests.post(facebook_post_url, data=facebook_post_data)

    if post_response.status_code != 200:
        print(f"Facebook post failed: {post_response.text}")
    else:
        print(f"Facebook post published successfully. Response: {post_response.text}")


def post_to_both(image_url, caption):
    """Simultaneously post to Instagram and Facebook."""
    print("Posting to Instagram...")
    post_to_instagram(image_url, caption)

    print("Posting to Facebook...")
    post_to_facebook(image_url, caption)


def post_images_from_folder(folder_path, time_interval):
    """Post multiple images from a folder to Instagram and Facebook with a time interval."""
    # Get a list of image files from the folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('jpg', 'jpeg', 'png'))]

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)

        # Generate an auto-caption for the image
        caption = generate_caption(image_path)

        # Upload image to Imgur and get the public URL
        image_url = upload_to_imgur(image_path)

        if image_url:
            print(f"Posting image: {image_path} with caption: '{caption}'")

            # Post the image to both Instagram and Facebook
            post_to_both(image_url, caption)

            # Wait for the defined time interval before posting the next image
            print(f"Waiting {time_interval} seconds before posting the next image...")
            time.sleep(time_interval)
        else:
            print(f"Skipping {image_file} due to Imgur upload failure.")


# Example usage
folder_path = '/content/image'  # Path to folder containing the images
time_interval = 20  # Time interval in seconds (e.g., 2 minutes)

post_images_from_folder(folder_path, time_interval)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Image uploaded to Imgur successfully. URL: https://i.imgur.com/EOm9bkg.jpeg
Posting image: /content/image/second.jpg with caption: 'a crescent in the sky with clouds and stars'
Posting to Instagram...
Instagram media uploaded successfully. Media ID: 18034656187992881
Instagram post published successfully. Response: {"id":"17879068635151147"}
Posting to Facebook...
Facebook post published successfully. Response: {"id":"122107976288533711","post_id":"460354680485224_122107976324533711"}
Waiting 20 seconds before posting the next image...
Image uploaded to Imgur successfully. URL: https://i.imgur.com/qUFCj63.jpeg
Posting image: /content/image/firstly.jpeg with caption: 'a wet wet road'
Posting to Instagram...
Instagram media uploaded successfully. Media ID: 17894335469999539
Instagram post published successfully. Response: {"id":"17876715825157511"}
Posting to Facebook...
Facebook post published successfully. Response: {"id":"122107976648533711","post_id":"460354680485224_1221079767025337